In [ ]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 13.3 MB/s eta 0:00:00


In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')
import catboost
from catboost import *
import sklearn
from sklearn.preprocessing import LabelEncoder
import sys

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/LG_Aimers_Phase_3/train.csv')
test = pd.read_csv('/content/drive/MyDrive/LG_Aimers_Phase_3/test.csv')
submit = pd.read_csv('/content/drive/MyDrive/LG_Aimers_Phase_3/sample_submission.csv')

In [ ]:
print(train[train['Y_Class'] == 0]['Y_Quality'].max())
print(train[train['Y_Class'] == 1]['Y_Quality'].min())
print(train[train['Y_Class'] == 1]['Y_Quality'].max())
print(train[train['Y_Class'] == 2]['Y_Quality'].min())

0.525066667
0.525085714
0.534842857
0.534950794


In [ ]:
def del_columns(train, test):
    col_list = train.columns
    nan_list = []
    nan_cnt = []
    nan_col = []
    full_list = []
    for col in col_list:
        if train[col].isnull().sum() == 0 :
            full_list.append(col)
            continue
        nan_list.append([col, train[col].isnull().sum()])
        nan_cnt.append(train[col].isnull().sum())
        nan_col.append(col)

    del_col = []
    for col in nan_list :
        if col[1] == len(train) :
            del_col.append(col[0])
    train = train.drop(columns=del_col)
    test = test.drop(columns=del_col)

    del_col = []
    col_list = train.describe().columns
    for col in col_list :
        if col == 'Y_Class':
            continue
        if col == 'Y_Quality':
            continue
        if col == 'LINE':
            continue
        if col == 'PRODUCT_CODE':
            continue
        if train[col].nunique()==1 :
            del_col.append(col)
    train = train.drop(columns=del_col)
    test = test.drop(columns=del_col)
    
    return train,test

def make_train_test_dataset(train,test):
    
    '''
    트레인데이터, 학습데이터 셋 만들기
    '''
    
    train_x = train.drop(columns=['PRODUCT_ID','PRODUCT_CODE','Y_Class','Y_Quality'])
    test_x = test.drop(columns=['PRODUCT_ID','PRODUCT_CODE'])
    train_y = train['Y_Quality']
    train_w = train[['Y_Class']]
    return train_x, test_x, train_y, train_w


def fillna(train,test,value):
    train = train.fillna(value)
    test = test.fillna(value)
    return train,test

def labelencoder(train,test,col_list):
    
    qual_col = col_list
    for i in qual_col:
        le = LabelEncoder()
        le = le.fit(train[i])
        train[i] = le.transform(train[i])

        for label in np.unique(test[i]): 
            if label not in le.classes_: 
                le.classes_ = np.append(le.classes_, label)
        test[i] = le.transform(test[i]) 
    return train,test

In [ ]:
#train, test = del_columns(train,test)
train, test = fillna(train,test,-999999)
#train_x, test_x, train_y, train_w = make_train_test_dataset(train,test)
#train_x, test_x = labelencoder(train_x,test_x,['LINE'])
train['PRODUCT_CODE'] = train['PRODUCT_CODE'].astype('category')
train['LINE'] = train['LINE'].astype('category')

test['PRODUCT_CODE'] = test['PRODUCT_CODE'].astype('category')
test['LINE'] = test['LINE'].astype('category')

In [ ]:
submit_r = pd.concat([submit,test['PRODUCT_CODE']],axis=1)

In [ ]:
train_A = train[train['PRODUCT_CODE'] == 'A_31']
train_T = train[train['PRODUCT_CODE'] == 'T_31']
train_O = train[train['PRODUCT_CODE'] == 'O_31']

test_A = test[test['PRODUCT_CODE'] == 'A_31']
test_T = test[test['PRODUCT_CODE'] == 'T_31']
test_O = test[test['PRODUCT_CODE'] == 'O_31']

submit_A = submit_r[submit_r['PRODUCT_CODE'] == 'A_31']
submit_T = submit_r[submit_r['PRODUCT_CODE'] == 'T_31']
submit_O = submit_r[submit_r['PRODUCT_CODE'] == 'O_31']

In [ ]:
train_A_1 = train_A[(train_A['X_230'] == -999999) & (train_A['X_231'] == -999999) & (train_A['X_232'] == -999999) & (train_A['X_233'] == -999999) & (train_A['X_234'] == -999999) & (train_A['X_235'] == -999999) & (train_A['X_236'] == -999999)]
train_T_1 = train_T[(train_T['X_3000'] == -999999) & (train_T['X_3001'] == -999999) & (train_T['X_3002'] == -999999) & (train_T['X_3003'] == -999999) & (train_T['X_3004'] == -999999) & (train_T['X_3005'] == -999999) & (train_T['X_3006'] == -999999)]
#train_O_1 = train_O[(train_O['X_93'] == -999999) & (train_O['X_94'] == -999999) & (train_O['X_95'] == -999999) & (train_O['X_96'] == -999999)]

train_A_2 = train_A[(train_A['X_230'] != -999999) & (train_A['X_231'] != -999999) & (train_A['X_232'] != -999999) & (train_A['X_233'] != -999999) & (train_A['X_234'] != -999999) & (train_A['X_235'] != -999999) & (train_A['X_236'] != -999999)]
train_T_2 = train_T[(train_T['X_3000'] != -999999) & (train_T['X_3001'] != -999999) & (train_T['X_3002'] != -999999) & (train_T['X_3003'] != -999999) & (train_T['X_3004'] != -999999) & (train_T['X_3005'] != -999999) & (train_T['X_3006'] != -999999)]
#train_O_2 = train_O[(train_O['X_93'] != -999999) & (train_O['X_94'] != -999999) & (train_O['X_95'] != -999999) & (train_O['X_96'] != -999999)]

test_A_1 = test_A[(test_A['X_230'] == -999999) & (test_A['X_231'] == -999999) & (test_A['X_232'] == -999999) & (test_A['X_233'] == -999999) & (test_A['X_234'] == -999999) & (test_A['X_235'] == -999999) & (test_A['X_236'] == -999999)]
test_T_1 = test_T[(test_T['X_3000'] == -999999) & (test_T['X_3001'] == -999999) & (test_T['X_3002'] == -999999) & (test_T['X_3003'] == -999999) & (test_T['X_3004'] == -999999) & (test_T['X_3005'] == -999999) & (test_T['X_3006'] == -999999)]
#test_O_1 = test_O[(test_O['X_93'] == -999999) & (test_O['X_94'] == -999999) & (test_O['X_95'] == -999999) & (test_O['X_96'] == -999999)]

test_A_2 = test_A[(test_A['X_230'] != -999999) & (test_A['X_231'] != -999999) & (test_A['X_232'] != -999999) & (test_A['X_233'] != -999999) & (test_A['X_234'] != -999999) & (test_A['X_235'] != -999999) & (test_A['X_236'] != -999999)]
test_T_2 = test_T[(test_T['X_3000'] != -999999) & (test_T['X_3001'] != -999999) & (test_T['X_3002'] != -999999) & (test_T['X_3003'] != -999999) & (test_T['X_3004'] != -999999) & (test_T['X_3005'] != -999999) & (test_T['X_3006'] != -999999)]
#test_O_2 = test_O[(test_O['X_93'] != -999999) & (test_O['X_94'] != -999999) & (test_O['X_95'] != -999999) & (test_O['X_96'] != -999999)]


In [ ]:
train_A_1_indices = train_A.index[(train_A['X_230'] == -999999) & (train_A['X_231'] == -999999) & (train_A['X_232'] == -999999) & (train_A['X_233'] == -999999) & (train_A['X_234'] == -999999) & (train_A['X_235'] == -999999) & (train_A['X_236'] == -999999)]
train_T_1_indices = train_T.index[(train_T['X_3000'] == -999999) & (train_T['X_3001'] == -999999) & (train_T['X_3002'] == -999999) & (train_T['X_3003'] == -999999) & (train_T['X_3004'] == -999999) & (train_T['X_3005'] == -999999) & (train_T['X_3006'] == -999999)]
#train_O_1_indices = train_O.index[(train_O['X_93'] == -999999) & (train_O['X_94'] == -999999) & (train_O['X_95'] == -999999) & (train_O['X_96'] == -999999)]

train_A_1 = train_A.loc[train_A_1_indices]
train_T_1 = train_T.loc[train_T_1_indices]
#train_O_1 = train_O.loc[train_O_1_indices]

train_A_2_indices = train_A.index[(train_A['X_230'] != -999999) | (train_A['X_231'] != -999999) | (train_A['X_232'] != -999999) | (train_A['X_233'] != -999999) | (train_A['X_234'] != -999999) | (train_A['X_235'] != -999999) | (train_A['X_236'] != -999999)]
train_T_2_indices = train_T.index[(train_T['X_3000'] != -999999) | (train_T['X_3001'] != -999999) | (train_T['X_3002'] != -999999) | (train_T['X_3003'] != -999999) | (train_T['X_3004'] != -999999) | (train_T['X_3005'] != -999999) | (train_T['X_3006'] != -999999)]
#train_O_2_indices = train_O.index[(train_O['X_93'] != -999999) | (train_O['X_94'] != -999999) | (train_O['X_95'] != -999999) | (train_O['X_96'] != -999999)]

train_A_2 = train_A.loc[train_A_2_indices]
train_T_2 = train_T.loc[train_T_2_indices]
#train_O_2 = train_O.loc[train_O_2_indices]

test_A_1_indices = test_A.index[(test_A['X_230'] == -999999) & (test_A['X_231'] == -999999) & (test_A['X_232'] == -999999) & (test_A['X_233'] == -999999) & (test_A['X_234'] == -999999) & (test_A['X_235'] == -999999) & (test_A['X_236'] == -999999)]
test_T_1_indices = test_T.index[(test_T['X_3000'] == -999999) & (test_T['X_3001'] == -999999) & (test_T['X_3002'] == -999999) & (test_T['X_3003'] == -999999) & (test_T['X_3004'] == -999999) & (test_T['X_3005'] == -999999) & (test_T['X_3006'] == -999999)]
#test_O_1_indices = test_O.index[(test_O['X_93'] == -999999) & (test_O['X_94'] == -999999) & (test_O['X_95'] == -999999) & (test_O['X_96'] == -999999)]

test_A_1 = test_A.loc[test_A_1_indices]
test_T_1 = test_T.loc[test_T_1_indices]
#test_O_1 = test_O.loc[test_O_1_indices]

test_A_2_indices = test_A.index[(test_A['X_230'] != -999999) | (test_A['X_231'] != -999999) | (test_A['X_232'] != -999999) | (test_A['X_233'] != -999999) | (test_A['X_234'] != -999999) | (test_A['X_235'] != -999999) | (test_A['X_236'] != -999999)]
test_T_2_indices = test_T.index[(test_T['X_3000'] != -999999) | (test_T['X_3001'] != -999999) | (test_T['X_3002'] != -999999) | (test_T['X_3003'] != -999999) | (test_T['X_3004'] != -999999) | (test_T['X_3005'] != -999999) | (test_T['X_3006'] != -999999)]
#test_O_2_indices = test_O.index[(test_O['X_93'] != -999999) | (test_O['X_94'] != -999999) | (test_O['X_95'] != -999999) | (test_O['X_96'] != -999999)]

test_A_2 = test_A.loc[test_A_2_indices]
test_T_2 = test_T.loc[test_T_2_indices]
#test_O_2 = test_O.loc[test_O_2_indices]

submit_A_1 = submit_A.loc[test_A_1_indices]
submit_T_1 = submit_T.loc[test_T_1_indices]
#submit_O_1 = submit_O.loc[test_O_1_indices]

submit_A_2 = submit_A.loc[test_A_2_indices]
submit_T_2 = submit_T.loc[test_T_2_indices]
#submit_O_2 = submit_O.loc[test_O_2_indices]


In [ ]:
train_A_1, test_A_1 = del_columns(train_A_1,test_A_1)
train_T_1, test_T_1 = del_columns(train_T_1,test_T_1)
#train_O_1, test_O_1 = del_columns(train_O_1,test_O_1)

train_A_2, test_A_2 = del_columns(train_A_2,test_A_2)
train_T_2, test_T_2 = del_columns(train_T_2,test_T_2)
#train_O_2, test_O_2 = del_columns(train_O_2,test_O_2)

train_O, test_O = del_columns(train_O,test_O)


In [ ]:
train_A_1_x, test_A_1_x, train_A_1_y, train_A_1_w = make_train_test_dataset(train_A_1,test_A_1)
train_A_2_x, test_A_2_x, train_A_2_y, train_A_2_w = make_train_test_dataset(train_A_2,test_A_2)

train_T_1_x, test_T_1_x, train_T_1_y, train_T_1_w  = make_train_test_dataset(train_T_1,test_T_1)
train_T_2_x, test_T_2_x, train_T_2_y, train_T_2_w  = make_train_test_dataset(train_T_2,test_T_2)

train_O_x, test_O_x, train_O_y, train_O_w = make_train_test_dataset(train_O,test_O)

#train_O_1_x, test_O_1_x, train_O_1_y, train_O_1_w = make_train_test_dataset(train_O_1,test_O_1)
#train_O_2_x, test_O_2_x, train_O_2_y, train_O_2_w = make_train_test_dataset(train_O_2,test_O_2)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
import copy
from itertools import product

def get_best_threshold_spliter(y_pred, y_true_cls, model=None, training=True):
    if training:
        #search_space = [[2, 4, 6, 18, 36], [1, 3, 5, 7, 9, 27, 54]]
        search_space = [[2, 4, 6], [1, 3, 5, 7, 9]]
        best_score = -np.inf
        output_pred = []
        model = None
        for depth, min_samples in product(*search_space): 
            model = DecisionTreeClassifier(
                criterion="gini", max_features=1.0,
                max_depth=depth, min_samples_leaf=min_samples, random_state=42
            )

            model.fit(y_pred, y_true_cls)
            y_pred_cls = model.predict(y_pred)
            score = metrics.f1_score(y_true_cls, y_pred_cls, average="macro")
            if ((best_score < score)):
                best_score = score
                print(f"Best score : {best_score}")
                output_pred = y_pred_cls.copy()
                model = copy.deepcopy(model)
        return model, output_pred
    else:
        output_pred = model.predict(y_pred)
        return output_pred

In [ ]:
A1_stats = pd.DataFrame({"qual": train_A_1_y, "cls": train_A_1_w["Y_Class"]}).groupby("cls")["qual"].describe()
A2_stats = pd.DataFrame({"qual": train_A_2_y, "cls": train_A_2_w["Y_Class"]}).groupby("cls")["qual"].describe()

T1_stats = pd.DataFrame({"qual": train_T_1_y, "cls": train_T_1_w["Y_Class"]}).groupby("cls")["qual"].describe()
T2_stats = pd.DataFrame({"qual": train_T_2_y, "cls": train_T_2_w["Y_Class"]}).groupby("cls")["qual"].describe()

O_stats = pd.DataFrame({"qual": train_O_y, "cls": train_O_w["Y_Class"]}).groupby("cls")["qual"].describe()

In [ ]:
O_stats

,count,mean,std,min,25%,50%,75%,max
cls,,,,,,,,
1,6.0,0.530601,0.002681,0.525916,0.529938,0.530954,0.532101,0.533702
2,2.0,0.535078,0.000180,0.534951,0.535014,0.535078,0.535141,0.535205


In [ ]:
#1
clf = catboost.CatBoostRegressor(
    learning_rate=0.05,# 0.05
    iterations=1000, #1000
    depth=6,
    l2_leaf_reg=5, #5
    border_count=254,cat_features=['LINE'],
    random_seed=313)
##################
clf.fit(train_A_1_x, train_A_1_y)
train_pred_A1 = clf.predict(train_A_1_x)
pred_A1 = clf.predict(test_A_1_x)

reg_prob = np.abs(np.stack([
    train_pred_A1 - A1_stats.loc[0, "mean"],
    train_pred_A1 - A1_stats.loc[1, "mean"],
    train_pred_A1 - A1_stats.loc[2, "mean"],
], axis=1))
model_threshold, train_pred_A1 = get_best_threshold_spliter(reg_prob, train_A_1_w["Y_Class"].values, training=True)

reg_prob = np.abs(np.stack([
    pred_A1 - A1_stats.loc[0, "mean"],
    pred_A1 - A1_stats.loc[1, "mean"],
    pred_A1 - A1_stats.loc[2, "mean"],
], axis=1))
pred_A1 = get_best_threshold_spliter(reg_prob, y_true_cls=None, model=model_threshold, training=False)
##################

##################
clf.fit(train_A_2_x, train_A_2_y)
train_pred_A2 = clf.predict(train_A_2_x)
pred_A2 = clf.predict(test_A_2_x)
reg_prob = np.abs(np.stack([
    train_pred_A2 - A2_stats.loc[0, "mean"],
    train_pred_A2 - A2_stats.loc[1, "mean"],
    train_pred_A2 - A2_stats.loc[2, "mean"],
], axis=1))
model_threshold, train_pred_A2 = get_best_threshold_spliter(reg_prob, train_A_2_w["Y_Class"].values, training=True)

reg_prob = np.abs(np.stack([
    pred_A2 - A2_stats.loc[0, "mean"],
    pred_A2 - A2_stats.loc[1, "mean"],
    pred_A2 - A2_stats.loc[2, "mean"],
], axis=1))
pred_A2 = get_best_threshold_spliter(reg_prob, y_true_cls=None, model=model_threshold, training=False)
##################




##################
clf.fit(train_T_1_x, train_T_1_y)
train_pred_T1 = clf.predict(train_T_1_x)
pred_T1 = clf.predict(test_T_1_x)
reg_prob = np.abs(np.stack([
    train_pred_T1 - T1_stats.loc[0, "mean"],
    train_pred_T1 - T1_stats.loc[1, "mean"],
    train_pred_T1 - T1_stats.loc[2, "mean"],
], axis=1))
model_threshold, train_pred_T1 = get_best_threshold_spliter(reg_prob, train_T_1_w["Y_Class"].values, training=True)

reg_prob = np.abs(np.stack([
    pred_T1 - T1_stats.loc[0, "mean"],
    pred_T1 - T1_stats.loc[1, "mean"],
    pred_T1 - T1_stats.loc[2, "mean"],
], axis=1))
pred_T1 = get_best_threshold_spliter(reg_prob, y_true_cls=None, model=model_threshold, training=False)
##################



##################
clf.fit(train_T_2_x, train_T_2_y)
train_pred_T2 = clf.predict(train_T_2_x)
pred_T2 = clf.predict(test_T_2_x)
reg_prob = np.abs(np.stack([
    train_pred_T2 - T2_stats.loc[0, "mean"],
    train_pred_T2 - T2_stats.loc[1, "mean"],
    train_pred_T2 - T2_stats.loc[2, "mean"],
], axis=1))
model_threshold, train_pred_T2 = get_best_threshold_spliter(reg_prob, train_T_2_w["Y_Class"].values, training=True)

reg_prob = np.abs(np.stack([
    pred_T2 - T2_stats.loc[0, "mean"],
    pred_T2 - T2_stats.loc[1, "mean"],
    pred_T2 - T2_stats.loc[2, "mean"],
], axis=1))
pred_T2 = get_best_threshold_spliter(reg_prob, y_true_cls=None, model=model_threshold, training=False)
##################
clf.fit(train_O_x, train_O_y)
train_pred_O = clf.predict(train_O_x)
pred_O = clf.predict(test_O_x)
reg_prob = np.abs(np.stack([
    train_pred_O - O_stats.loc[1, "mean"],
    train_pred_O - O_stats.loc[2, "mean"],
], axis=1))
model_threshold, train_pred_O = get_best_threshold_spliter(reg_prob, train_O_w["Y_Class"].values, training=True)

reg_prob = np.abs(np.stack([
    pred_O - O_stats.loc[1, "mean"],
    pred_O - O_stats.loc[2, "mean"],
], axis=1))
pred_O = get_best_threshold_spliter(reg_prob, y_true_cls=None, model=model_threshold, training=False)
##################



In [ ]:
clf.fit(train_O_x, train_O_y)
train_pred_O = clf.predict(train_O_x)
pred_O = clf.predict(test_O_x)
reg_prob = np.abs(np.stack([
    #train_pred_O - O_stats.loc[0, "mean"],
    train_pred_O - O_stats.loc[1, "mean"],
    train_pred_O - O_stats.loc[2, "mean"],
], axis=1))
model_threshold, train_pred_O = get_best_threshold_spliter(reg_prob, train_O_w["Y_Class"].values, training=True)

reg_prob = np.abs(np.stack([
    #pred_O - O_stats.loc[0, "mean"],
    pred_O - O_stats.loc[1, "mean"],
    pred_O - O_stats.loc[2, "mean"],
], axis=1))
pred_O = get_best_threshold_spliter(reg_prob, y_true_cls=None, model=model_threshold, training=False)
##################


In [ ]:
submit_A_1['Y_Class'] = pred_A1
submit_A_2['Y_Class'] = pred_A2

submit_T_1['Y_Class'] = pred_T1
submit_T_2['Y_Class'] = pred_T2

submit_O['Y_Class'] = pred_O

In [ ]:
submit_combined = pd.concat([submit_A_1, submit_A_2, submit_T_1, submit_T_2, submit_O], axis=0)
submit_combined = submit_combined.reset_index(drop=True)
submit_combined = submit_combined.sort_values('PRODUCT_ID')
submit_combined.drop(['PRODUCT_CODE'],axis=1,inplace=True)
submit_combined = submit_combined.reset_index(drop=True)
submit_combined.to_csv('upgrade1_dt.csv',index=False)
submit_combined['Y_Class'].value_counts()

1    414
2     76
0     45
Name: Y_Class, dtype: int64